In [1]:
import os 
import string
from numpy import loadtxt
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

from collections import Counter

import warnings
warnings.filterwarnings('ignore')
import sys, os
sys.path.append(os.pardir)
import glob
import numpy as np


from multiprocessing import Pool
import multiprocessing as multi
sys.path.append(os.pardir)
import pandas as pd
import h5py
import re
import gensim

# lines = loadtxt("/home/louis/SharedWindows/edu_data/"+"imdb-edus.train")
# train = []

# with open(dir+"imdb-edus.train", 'r') as train_file:
#     lines = train_file.readlines()


##



# '''text2index'''
# 
# from func import TextDeal
# from func import wrapper_func_four
# 
# vocab = wList
# 
# td = TextDeal()
# load_path = sorted( glob.glob( '/home/dl-box/デスクトップ/PythonFile/hirose/Text/text_list/*.txt' ) )
# name = [ os.path.splitext( os.path.basename( r ) )[0] for r in load_path ]
# save_path = '/home/dl-box/デスクトップ/PythonFile/hirose/LSTM/W2I/'



/home/owner/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/owner/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
def wrapper_func_six(tuple_data):
    return(tuple_data[0](tuple_data[1],tuple_data[2],tuple_data[3],tuple_data[4], tuple_data[5], tuple_data[6]) )

def wrapper_func_eight(tuple_data):
    return(tuple_data[0](tuple_data[1],tuple_data[2],tuple_data[3],tuple_data[4], tuple_data[5], tuple_data[6], tuple_data[7], tuple_data[8]) )

def readEduStyledFiles(dir, filename):
    with open(dir+filename, 'r') as train_file:
        lines = train_file.readlines()
    return lines
        
def tokenizeListOfTexts(list, vocab):
    idx = []
    isEmpty = False
    for word in list:
        if word in vocab:
            idx.append(vocab[word].index)
            #idx.append(len(word))
            #print(word)
        # else:
        #     print(word)
    if idx == []:
        isEmpty = True
    return idx, isEmpty

def addUniqueElementsToList(docs, li):
    for doc in docs:
        for sentence in doc:
            for word in sentence:
                if word not in li:
                    #print(word)
                    li.append(word)
    return li

def generateUniqueIndices(path_list):
    vocab_list = []
    for filepath in path_list:
        tmp = np.load(filepath)
        li = addUniqueElementsToList(tmp, vocab_list)
        vocab_list = vocab_list + li
    return vocab_list


def cleanAndTokenizeEDUs(vocab, load_path=None, load_name=None, save_path=None, save_name=None, willReturn = True, max_sent_len = 40,max_doc_len = 15):
    
    if (willReturn == False) and (all(v is None for v in [load_path, load_name, save_path, save_name])):
        raise ValueError('Function Arguments cannot all be None Type when saving files')
    elif (willReturn == True) and (all(v is None for v in [load_path, load_name])):
        raise ValueError('Load arguments cannot all be None Type when loading file')
    
    lines = readEduStyledFiles(load_path,load_name)

    cleaned_lines = []
    length = len(lines)
    scores = []
    ids = []
    cleaned_doc=[]
    cleaned_all=[]
    idx_binary = [] # index for binary training only
    scores_binary =[]
    
    doc_count = 0
    sent_counter = -1
    
    for i in range(len(lines)):  
        
        if len(cleaned_all) in [114,116,135]:
            a = 3
               
        sent_counter += 1
        
        cleaned = lines[i].replace('\n','').replace('<s>','').replace("'",'')
        cleaned = text_to_word_sequence(cleaned)
        
#         if i in [91,92,94]:
#             print(str(i)+' now enters')
#             print(cleaned)
#             print(sent_counter)
        
        if (len(cleaned_doc) <= max_doc_len):


#                 #DEBUGCODE
#                 if i in [91,92,94]:
#                     print(str(i)+' exit at len = 0 for sent<max')
                
            if (len(cleaned) == 2): # and (i != 0):
#                 #DEBUGCODE
#                 if i in [91,92,94]:
#                     print(str(i)+' exit at len = 2 for sent<max')
                
                if  (cleaned[0].isnumeric() and cleaned[1].isnumeric()):
                    
                    if i != 0:
                        if(len(cleaned_doc) < max_doc_len):
                            while (len(cleaned_doc) < max_doc_len):
                                tokenized = [0]
                                cleaned_doc.append(tokenized)
                    
                    score = int(cleaned[0])
                    id = int(cleaned[1])
                    scores.append(score)
                    ids.append(id)
                    
                    if (score<=4) or (score>=7):
                        idx_binary.append(doc_count)
                        if (score<4):
                            scores_binary.append(0)
                        else:
                            scores_binary.append(1)
                    
                    if (i != 0):
                        filtered_doc = list(filter(None, cleaned_doc))
                        padded_doc = pad_sequences(filtered_doc, maxlen=max_sent_len)

                        cleaned_all.append(padded_doc.tolist())
                        
                        #DEBUGCODE
                        if len(cleaned_all) in [114,116,135]:
                            print(str(len(cleaned_all))+' exit at len=2 for sent<=max') 
                        cleaned_doc=[]

                    doc_count += 1
                    sent_counter = 0
                   
                    continue
#             #DEBUGCODE        
#             if i in [91,92,94]:
#                 print(str(i)+' exit at all len for sent<max')
            if (len(cleaned_doc) <= max_doc_len-1):
                if cleaned != []:
                    tokenized, isEmpty =tokenizeListOfTexts(cleaned, vocab)
                    if not isEmpty:
                        cleaned_doc.append(tokenized)
            
        else:
            if (len(cleaned) == 2): # and (i != 0):
#                 #DEBUGCODE
#                 if i in [91,92,94]:
#                     print(str(i)+' exit at len = 2 for sent=>max')
                
                if  (cleaned[0].isnumeric() and cleaned[1].isnumeric()):
                    
                    score = int(cleaned[0])
                    id = int(cleaned[1])
                    scores.append(score)
                    ids.append(id)
                    
                    if (score<=4) or (score>=7):
                        idx_binary.append(doc_count)
                        if (score<4):
                            scores_binary.append(0)
                        else:
                            scores_binary.append(1)
                    
                    if (i != 0):
                        filtered_doc = list(filter(None, cleaned_doc))
                        padded_doc = pad_sequences(filtered_doc, maxlen=max_sent_len)
                        
                        cleaned_all.append(padded_doc.tolist())
                        
                        #DEBUGCODE
                        if len(cleaned_all) in [114,116,135]:
                            print(str(len(cleaned_all))+' exit at len=2 for sent>max') 
                            print(cleaned_doc)
                        cleaned_doc=[]
                        doc_count += 1
                        
                    sent_counter = 0
                    
#             #DEBUGCODE
#             if i in [91,92,94]:
#                 print(str(i)+' exit at all len for sent=>max') 
#                 print(sent_counter)
            
        if (i%1000000 == 0) and (i!=0):
            print(str(i)+' out of' + str(length))
            #DEBUG CODE
            #break
            # print(cleaned_lines[i-500:i])
            

    
    # Append the last doc 
    
    while (sent_counter < max_doc_len):
        tokenized = [0]
        cleaned_doc.append(tokenized)
        sent_counter += 1
        
    filtered_doc = list(filter(None, cleaned_doc))
    padded_doc = pad_sequences(filtered_doc, maxlen=max_sent_len)
    
    cleaned_all.append(padded_doc.tolist())
        
        # cleaned_doc = cleaned_doc[0]
    # 
    # del cleaned_all[0]
    # del scores[0]
    # del idx_binary[0]
    # del ids[0]
    features = np.asarray(cleaned_all)
    labels = np.asarray(scores)
    idx = np.asarray(idx_binary)
    
   
    
    if (willReturn == True):
        print('cleanAndTokenizeEDUs Returned and Done')
        return features, labels, idx, ids
    else:
        np.save(save_path + save_name + '_features'+ '.npy', features)
        np.save(save_path + save_name + '_labels' + '.npy', labels)
        np.save(save_path + save_name + '_idx_binary_only' + '.npy', idx)
        np.save(save_path + save_name + '_doc_ids' + '.npy', ids)
        np.save(save_path + save_name + '_scores_binary' + '.npy', scores_binary)


In [3]:
'''isReload = int(input("Type 1 to initiate reload of google word2vec: "))
    
if isReload:
    print('word2vec model now loading')
    model_word2vec_temp = gensim.models.KeyedVectors.load_word2vec_format('/home/owner/デスクトップ/milnet+edu/GoogleNews-vectors-negative300.bin', binary=True)  
    model_word2vec = model_word2vec_temp
    print('word2vec model loaded')

load_path = '/home/owner/デスクトップ/milnet+edu/data/'  
save_path = '/home/owner/デスクトップ/milnet+edu/data/Preprocessed/'

load_name = ['imdb-edus.train', 'imdb-edus.test','imdb-edus.dev']
save_name = ['train_data','test_data','validation_data']
name_trailers = ['_features','_labels','_idx_binary_only','_doc_ids']
willReturn = True

vocab = model_word2vec.vocab

max_sent_len = 15
max_doc_len = 40

features, scores, idx, ids = cleanAndTokenizeEDUs(vocab, load_path=load_path, load_name = 'imdb-edus.train', save_path=save_path,save_name=save_name[0], 
                                                  willReturn = willReturn, max_sent_len=max_sent_len, max_doc_len=max_doc_len)


i=0
#cleanAndTokenizeEDUs(model_word2vec.vocab, load_path=load_path, load_name = load_name[i], save_path=save_path,save_name=save_name[i], willSave = False)'''

Type 1 to initiate reload of google word2vec: 1
word2vec model now loading


KeyboardInterrupt: 

In [27]:
'''features = x_valid
print(len(features))
for i in range(len(features)):
    #print(len(features[i]))
    if len(features[i])!=40:
            print('i:'+str(i)+'   ;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;')
            print(len(features[i]))
            x_valid[i].append(x_valid[i][-1])
    for j in range(len(features[i])):
        
        #print(len(features[i][j]))
        if len(features[i][j])!= 15:
            print('i:'+str(i)+'j:'+str(j)+'   ************************************************************')

x_valid=np.asarray(x_valid.tolist())
##'''

25206


In [5]:
if __name__ == '__main__':
    #data=(関数, 引数)
    isReload = int(input("Type 1 to initiate reload of google word2vec: "))
    
    if isReload:
        print('word2vec model now loading')
        model_word2vec_temp = gensim.models.KeyedVectors.load_word2vec_format('/home/owner/デスクトップ/milnet+edu/GoogleNews-vectors-negative300.bin', binary=True)  
        model_word2vec = model_word2vec_temp

    print('word2vec model loaded')

    load_path = '/home/owner/デスクトップ/milnet+edu/data/'  
    save_path = '/home/owner/デスクトップ/milnet+edu/data/Preprocessed/20words+50sent/'
      
    load_name = ['imdb-edus.train', 'imdb-edus.test','imdb-edus.dev']
    save_name = ['train_data','test_data','validation_data']
    name_trailers = ['_features','_labels','_idx_binary_only','_doc_ids']
    willReturn = False
    
    max_sent_len = 15
    max_doc_len = 40

    #for i in range(len(load_name)):
    #i = 0
    #a,b,c,d = cleanAndTokenizeEDUs(model_word2vec.vocab, load_path=load_path, load_name = load_name[i], save_path=save_path,save_name=save_name[i], willSave = False)
    
    # features, scores, idx, ids = cleanAndTokenizeEDUs(model_word2vec.vocab, load_path=load_path, load_name = 'imdb-edus.train', save_path=save_path,save_name=save_name[0], willReturn = False)
    
    vocab = model_word2vec.vocab
    data = [ ( cleanAndTokenizeEDUs, vocab, load_path, load_name[i], save_path, save_name[i] , willReturn, max_sent_len, max_doc_len) for i in range( len(load_name) ) ]
    with Pool( multi.cpu_count()-1 ) as p:
        p.map( wrapper_func_eight, data )
    


Type 1 to initiate reload of google word2vec: 0
word2vec model loaded
114 exit at len=2 for sent<=max
135 exit at len=2 for sent<=max
116 exit at len=2 for sent<=max
114 exit at len=2 for sent<=max
116 exit at len=2 for sent<=max
135 exit at len=2 for sent<=max
114 exit at len=2 for sent<=max
116 exit at len=2 for sent<=max
135 exit at len=2 for sent<=max
1000000 out of11039529
1000000 out of1349298
1000000 out of1335054
2000000 out of11039529
3000000 out of11039529
4000000 out of11039529
5000000 out of11039529
6000000 out of11039529
7000000 out of11039529
8000000 out of11039529
9000000 out of11039529
10000000 out of11039529
11000000 out of11039529


In [7]:
print('Starting Vocab List')
docs = [save_path+name+'_features'+'.npy' for name in save_name]
vocab_list = generateUniqueIndices(docs)
print('finished generating vocab')

Starting Vocab List
finished generating vocab


In [8]:
len(vocab_list)

492344

In [9]:
vocab_list = sorted(set(vocab_list))
np.save(save_path + 'vocab_idx.npy', vocab_list)

print('All Done')

All Done


In [10]:
np.save(save_path + 'weights.npy', model_word2vec.wv.syn0[vocab_list])    
print('Weights Saved')

Weights Saved


In [5]:
'''
train_features = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/train_data_features.npy')
valid_features = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/validation_data_features.npy')

test_features = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/test_data_features.npy')
test_labels = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/test_data_labels.npy')


valid_features = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/validation_data_features.npy')'''

In [8]:
'''print(valid_features)
'''

[list([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 33, 16, 4129, 4501], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 58, 103832, 1165, 643], [653, 11, 3172, 11, 1422, 4441, 219, 3572, 4501, 336, 31165, 1133, 5, 572747, 24914], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4501, 297, 158, 28], [0, 0, 0, 0, 28, 1121, 900, 183, 103832, 21, 150, 1, 11, 132, 9729], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 86, 712, 170, 10473], [0, 0, 0, 0, 3, 28, 1121, 183, 103832, 21, 673, 1, 11, 132, 9729], [0, 0, 0, 0, 0, 0, 0, 0, 28, 47, 89, 16, 2618, 18, 68], [685, 189, 9357, 159, 67480, 4501, 31165, 468, 11, 84855, 1280, 56, 2119, 5, 1973], [0, 0, 0, 0, 0, 0, 33, 122, 525, 80238, 314419, 572747, 2579, 28, 1121], [0, 0, 0, 0, 0, 0, 0, 0, 900, 12, 11, 629, 11833, 109483, 22163], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 4123, 4], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 117, 572747, 3798], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 11, 132, 11, 1002, 271], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 162, 152, 95, 26, 1643], [0, 0, 0, 0, 0, 0, 0, 0, 28, 84855, 4, 13, 11, 2

In [14]:
'''output = set()
trends = [1,2,6,8,9,7,6,5,4,3,4,5,]
for x in trends:
    output.add(x)
print (output)'''
vocab_list

array([      0,       1,       2, ..., 2991871, 2993225, 2997429])

In [11]:
print(len(model_word2vec_temp.wv.syn0[vocab_list]))
print(len(vocab_list))

61628
61628


In [15]:
vocab_list = np.asarray(vocab_list)
np.save(save_path + save_name + '_word_indices' + '.npy', vocab_list)

TypeError: must be str, not list

In [16]:
x_train = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/20words+50sent/train_data_features.npy')#np.load('/home/owner/デスクトップ/PythonFile/imdb/x_train_sort.npy')
y_train = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/20words+50sent/train_data_scores_binary.npy')#np.load('/home/owner/デスクトップ/PythonFile/imdb/x_test_sort.npy')


x_test = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/20words+50sent/test_data_features.npy')#np.load('/home/owner/デスクトップ/PythonFile/imdb/t_train.npy')
y_test = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/20words+50sent/test_data_scores_binary.npy')#np.load('/home/owner/デスクトップ/PythonFile/imdb/t_test.npy')

x_valid = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/20words+50sent/validation_data_features.npy')
y_valid = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/20words+50sent/validation_data_scores_binary.npy')

train_idx = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/20words+50sent/train_data_idx_binary_only.npy')
test_idx = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/20words+50sent/test_data_idx_binary_only.npy')
valid_idx = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/20words+50sent/validation_data_idx_binary_only.npy')

#print(len(train_idx))
x_train = x_train[train_idx]
#y_train = y_train[train_idx]
#print(len(x_train))
#print(len(y_train))

x_test = x_test[test_idx]
#y_test = y_test[test_idx]

x_valid = x_valid[valid_idx]
#y_valid = y_valid[valid_idx]

vocab_list = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/20words+50sent/vocab_idx.npy')

In [17]:
import collections

flat_list = vocab_list#[word for doc in x_train for sent in doc for word in sent]
counts = collections.Counter(flat_list)

sorted_keys = sorted(counts.keys())
li = [x for x in range(len(sorted_keys))]
print(len(sorted_keys), len(li))

mapping = dict(zip(sorted_keys,li))
print(mapping)

61628 61628
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 8: 6, 9: 7, 10: 8, 11: 9, 12: 10, 13: 11, 14: 12, 15: 13, 16: 14, 17: 15, 18: 16, 19: 17, 21: 18, 22: 19, 23: 20, 24: 21, 26: 22, 27: 23, 28: 24, 29: 25, 30: 26, 31: 27, 32: 28, 33: 29, 35: 30, 36: 31, 37: 32, 38: 33, 39: 34, 41: 35, 42: 36, 43: 37, 44: 38, 45: 39, 46: 40, 47: 41, 48: 42, 49: 43, 50: 44, 52: 45, 53: 46, 54: 47, 55: 48, 56: 49, 58: 50, 59: 51, 60: 52, 61: 53, 63: 54, 64: 55, 65: 56, 66: 57, 67: 58, 68: 59, 69: 60, 71: 61, 72: 62, 74: 63, 75: 64, 76: 65, 77: 66, 78: 67, 80: 68, 81: 69, 82: 70, 83: 71, 85: 72, 86: 73, 87: 74, 88: 75, 89: 76, 90: 77, 91: 78, 92: 79, 93: 80, 94: 81, 95: 82, 96: 83, 97: 84, 99: 85, 100: 86, 101: 87, 102: 88, 103: 89, 104: 90, 106: 91, 107: 92, 108: 93, 109: 94, 110: 95, 111: 96, 112: 97, 113: 98, 114: 99, 115: 100, 116: 101, 117: 102, 118: 103, 119: 104, 120: 105, 121: 106, 122: 107, 124: 108, 125: 109, 126: 110, 127: 111, 130: 112, 131: 113, 132: 114, 133: 115, 134: 116, 135: 117, 136: 118, 1

In [18]:
def wrapper_func_three(tuple_data):
    return(tuple_data[0](tuple_data[1],tuple_data[2],tuple_data[3]) )

def replace_matched_items(docs, dictionary, save_name):
    print('started '+ save_name)
    features = [[[dictionary.get(word, word) for word in sent] for sent in doc] for doc in docs]
    np.save('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/20words+50sent/' + save_name + '_features_sorted'+ '.npy', features)
    print('finished '+ save_name)
#list_ = replace_matched_items(x_test.tolist(), mapping)

In [19]:
variables=[x_train, x_test, x_valid]
save_name = ['train_data','test_data','validation_data']

data = [ ( replace_matched_items, variables[i], mapping, save_name[i]) for i in range( len(variables) ) ]
with Pool( multi.cpu_count()-1 ) as p:
    p.map( wrapper_func_three, data )

started test_data
started validation_data
finished test_data
finished validation_data
started train_data
finished train_data


In [36]:
embWeights=np.load( '/home/owner/デスクトップ/milnet+edu/data/Preprocessed/' + 'weights.npy')#np.load('/home/owner/デスクトップ/PythonFile/imdb/weights.npy')


In [30]:
x_train

array([[[      0,       0,       0, ...,     125,     331,     200],
        [      0,       0,       0, ...,      33,      61,    1642],
        [      0,       0,       0, ...,     165,    2040,    7276],
        ...,
        [      0,       0,       0, ...,       0,       0,       0],
        [      0,       0,       0, ...,       0,       0,       0],
        [      0,       0,       0, ...,       0,       0,       0]],

       [[      0,       0,       0, ...,       1,      26,     512],
        [      0,       0,       0, ...,    4501,   31165,    3894],
        [      0,       0,       0, ...,       2,   70882,    8352],
        ...,
        [      0,       0,       0, ...,       0,       0,       0],
        [      0,       0,       0, ...,       0,       0,       0],
        [      0,       0,       0, ...,       0,       0,       0]],

       [[      0,       0,       0, ...,   10626,     943,     172],
        [      0,       0,       0, ...,      27,    7276,   36706],
    

In [50]:
x_tr_new =np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/' + save_name[2] + '_features_sorted'+ '.npy')


In [51]:
docs = x_tr_new
#max_idx= 0
for doc in docs:
    for sent in doc:
        for word in sent:
            if int(word)>max_idx:
                max_idx = int(word)

In [52]:
print(max_idx,len(embWeights))

61559 61560
